In [1]:
%run src/flask_import.py

In [2]:
import pandas as pd

In [4]:
flaskdf = pd.read_pickle('data/flaskdf.pkl')
locations_list = flaskdf.LOCATION.unique()
locations_list.sort()
locations_list

In [11]:
flaskdf = pd.read_pickle('data/flaskdf.pkl')
zips_list = flaskdf['ZIP OR POSTAL CODE'].unique()
zips_list.sort()
zips_list

array([78002, 78009, 78023, 78026, 78052, 78059, 78064, 78065, 78069,
       78073, 78101, 78109, 78113, 78114, 78121, 78147, 78148, 78152,
       78154, 78161, 78163, 78199, 78201, 78202, 78203, 78204, 78207,
       78208, 78209, 78210, 78211, 78212, 78213, 78214, 78215, 78216,
       78217, 78218, 78219, 78220, 78221, 78222, 78223, 78224, 78225,
       78226, 78227, 78228, 78230, 78231, 78232, 78233, 78234, 78239,
       78240, 78242, 78244, 78245, 78247, 78248, 78249, 78250, 78251,
       78253, 78254, 78255, 78256, 78257, 78258, 78259, 78260, 78261,
       78263, 78264])

In [34]:
%%writefile flaskapp1.py

from flask import Flask, request, render_template, url_for, redirect
import pickle
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from io import BytesIO
from datetime import date

# Initialize app
app = Flask(__name__)

# load the pickled model
with open('data/model_rfr_full.pkl', 'rb') as f:
    model = pickle.load(f)
    
flaskdf = pd.read_pickle('data/flaskdf.pkl')
locations_list = flaskdf.LOCATION.unique()
locations_list.sort()

zips_list = flaskdf['ZIP OR POSTAL CODE'].unique()
zips_list.sort()


# # load the pickled training data to display with prediction
# with open('data/trainXY.pkl', 'rb') as f:
#     trainXY = pickle.load(f)

# trainX = trainXY[:,:2]
# trainY = trainXY[:,2]

# Home page with form on it to submit new data
@app.route('/')
def get_new_data():
    

    flaskdf = pd.read_pickle('data/flaskdf.pkl')
    locations_list = flaskdf.LOCATION.unique()
    locations_list.sort()
    #     zips = zips_list
    return render_template('dropdown.html', locations_list=locations_list)
#        '''
#         <form action="/predict" method='POST'>
#           House Address:
#           <br>
#           <input type="str" name="Address"> 
#           <br>
#           Bedrooms:
#           <br>
#           <input type="int" name="Bedrooms"> 
#           <br>
#           Bathrooms:
#           <br>
#           <input type="text" name="Bathrooms"> 
#           <br>
#           Neighborhood:
#           <br>
#           <input type="text" name="Neighborhood"> 
#           <br>
#           Zip Code:
#           <br>
#           <input type="text" name="Zip Code"> 
#           <br>
#           Square Feet:
#           <br>
#           <input type="text" name="Square Feet"> 
#           <br>
#           Year Built:
#           <br>
#           <input type="text" name="Year"> 
#           <br>
#           Lot Size:
#           <br>
#           <input type="text" name="Lot Size"> 
#           <br>
#           <br>
#           <input type="submit" value="Submit for house price estimation">
#         </form>
#         '''
    

# def dropdown():
#     flaskdf = pd.read_pickle('data/flaskdf.pkl')
#     locations_list = flaskdf.LOCATION.unique()
#     locations_list.sort()
# #     zips = zips_list
#     return render_template('dropdown.html', locations_list=locations_list)

# @app.route('/dropdown', methods=['GET'])
# def dropdown():
#     neighborhoods = locations_list
#     zips = zips_list
#     return render_template('dropdown.html', locations_list=locations_list)

@app.route('/predict', methods = ["GET", "POST"])
def predict():
    # request the text from the form 
    Address = str(request.form['Address'])
    Bedrooms = float(request.form['Bedrooms'])
    Bathrooms = float(request.form['Bathrooms'])
    # Neighborhood = float(request.form['Neighborhood'])
    Zip = float(request.form['Zip Code'])
    SqFt = float(request.form['Square Feet'])
    Year = float(request.form['Year'])
    Lot = float(request.form['Lot Size'])
    Month = date.today().month
    Yeartoday = date.today().year
    #Placeholders for latitude and longitude and HOA
    Lat = 29.483
    Long = -98.51
    HOA = 0 
    X_n = np.array([[Zip, SqFt, Lot, Year, HOA, Lat, Long, (Bedrooms+Bathrooms), Month,Yeartoday]])
    
    # predict on the new data
    Y_pred = model.predict(X_n)

    '''
    # for plotting 
    X_0 = trainX[trainY == 0] # class 0
    X_1 = trainX[trainY == 1] # class 1
    X_2 = trainX[trainY == 2] # class 2
    
    # color-coding prediction 
    if Y_pred[0] == 0:
        cp = 'b'
    elif Y_pred[0] == 1:
        cp = 'r'
    else:
        cp = 'g'

    if plt:
        plt.clf() # clears the figure when browser back arrow used to enter new data

    plt.scatter(X_0[:, 0], X_0[:, 1], c='b', edgecolors='k', label = 'class 0')
    plt.scatter(X_1[:, 0], X_1[:, 1], c='r', edgecolors='k', label = 'class 1')
    plt.scatter(X_2[:, 0], X_2[:, 1], c='g', edgecolors='k', label = 'class 2')
    plt.scatter(X_n[:, 0], X_n[:, 1], c=cp, edgecolors='k', marker = 'd', \
        s=100, label = 'prediction')
    plt.xlabel('Sepal length')
    plt.ylabel('Sepal width')
    plt.title('Prediction plotted with training data')
    plt.legend()
        
    image = BytesIO()
    plt.savefig(image)
    out = image.getvalue(), 200, {'Content-Type': 'image/png'}
    '''
    
    out = str(Y_pred)
    
    return '${}'.format(out)

if __name__ == '__main__':
    app.run(host='0.0.0.0', port=8080, debug=True)

Overwriting flaskapp1.py
